In [ ]:
#This script creates freesurfer commands for recon-all in the specified directories.

import os
def init_freesurfer_command(FSHOME_PATH, MRI_DIR, nobash=True):
    if nobash:
        command1 = ""
    else:
        command1 = "bash\n"
    command2 = "export FREESURFER_HOME=" + FSHOME_PATH + "\n"
    command3 = "source $FREESURFER_HOME/SetUpFreeSurfer.sh\n"
    command4 = "export SUBJECTS_DIR=" + MRI_DIR + "\n\n"
    return command1 + command2 + command3 + command4

def make_freesurfer_command_recon_all(T1w_path, T2w_path, subject_name):
    #recon-all -subject subjectname -i /path/to/input_volume -T2 /path/to/T2_volume -T2pial -all
    my_subject = "my_subject=" + subject_name + "\n"
    my_t1 = "my_t1=" + T1w_path + "\n"
    if T2w_path is not None:
        my_t2 = "my_t2=" + T2w_path + "\n"
        recon_all_command = "recon-all -subject $my_subject -i $my_t1 -T2 $my_t2 -T2pial -all\n\n"
    else:
        my_t2 = ""
        recon_all_command = "recon-all -subject $my_subject -i $my_t1 -all\n\n"
    return my_subject + my_t1 + my_t2 + recon_all_command

def bem_freesurfer_commands_for_subject(reftep_reco_dir,subject_name): #not used here.
    my_subject = "my_subject=" + subject_name + "\n"
    bemdir = "bemdir=" + reftep_reco_dir + subject_name + '/bem/' + "\n"
    mkbemdir = "mkdir $bemdir\n"
    t1_mgz = "t1_mgz=" + reftep_reco_dir + subject_name + '/mri/' + 'T1.mgz' + "\n"
    mri_watershed_command = "mri_watershed -atlas -useSRAS -surf $bemdir $t1_mgz $bemdir" + "\n"
    mne_scalp_surfaces = "mne make_scalp_surfaces --subject $my_subject --force" + "\n\n"
    return my_subject + bemdir + mkbemdir + t1_mgz + mri_watershed_command + mne_scalp_surfaces

In [ ]:
filename = "recon_all_freesurfer_commands_reftep_aalto_tue.sh"
freesurfer_home = ""

reftep_reco_dir_aalto = ""
reftep_reco_dir_tue = ""
reftep_reco_dir_aalto_fatsuppressed = ""

subjects_path_aalto = ""
subjects_path_tue = ""


In [ ]:
with open(filename, "w") as freesurfer_command_file:
    #write the initialization commands to make running freesurfer possible
    initcommand = init_freesurfer_command(freesurfer_home, reftep_reco_dir_aalto,nobash=True)
    freesurfer_command_file.write(initcommand)
    sub_dirs = os.listdir(subjects_path_aalto)
    for subject in sub_dirs:
        if subject != '.DS_Store':
            T1w_path, T2w_path, mri_directory_correct = None, None, None #for control and non overlapping structs
            subject_path_now = subjects_path_aalto + subject + "/"
            session_types = os.listdir(subject_path_now)[1:]
            for session_type in session_types:
                directory_here = subject_path_now + session_type + "/"
                dirs_here = os.listdir(directory_here)
                if 'anat' in dirs_here and 'dwi' in dirs_here:
                    mri_dirs_in_mri_dir = dirs_here
                    mri_directory = directory_here
            if mri_dirs_in_mri_dir is not None:
                print("MRIs found for " + subject)
                for mri_dir in mri_dirs_in_mri_dir:
                    if mri_dir == "anat":
                        anat_dir = mri_directory + mri_dir + "/"
                        print(anat_dir)
                        for recording_type in os.listdir(anat_dir):
                            if "T1w.nii" in recording_type and "fatsuppressed" not in recording_type:
                                cdir = "".join([subjects_path_aalto ,"".join(anat_dir.split("BIDS")[1][1:])])
                                T1w_path = cdir + recording_type
                            if "T2w.nii" in recording_type:
                                cdir = "".join([subjects_path_aalto ,"".join(anat_dir.split("BIDS")[1][1:])])
                                T2w_path = cdir + recording_type
                if T1w_path is not None and T2w_path is not None:
                    subject_name = subject.split("-")[0] + "-" + subject.split("-")[1][-3:]+ "_reco"
                    freesurfer_commands_for_subject = make_freesurfer_command_recon_all(T1w_path, T2w_path, subject_name)
                    freesurfer_command_file.write(freesurfer_commands_for_subject)
                    print("Recon-all commands made for subject " + subject_name +"\n")
            else:
                print("Did not find MRIs for "+ subject)

    initcommand_tue = init_freesurfer_command(freesurfer_home, reftep_reco_dir_tue, nobash=True) #separately for tuebingen, no bash command needed because same file
    freesurfer_command_file.write(initcommand_tue)
    tue_subdirs = os.listdir(subjects_path_tue)
    for subject in tue_subdirs:
        T1w_path, T2w_path = None, None #for control and non overlapping structs
        filepaths = subjects_path_tue + "/" + subject + "/"
        for file in os.listdir(filepaths):
            if file.endswith("T1w_MPR_7.nii.gz"):
                cpath = filepaths + "/" + file
                T1w_path = "".join([subjects_path_tue,subject,"/",file])
            if file.endswith("T2w_SPC_9.nii.gz"):
                cpath = filepaths + "/" + file
                T2w_path = "".join([subjects_path_tue,subject,"/",file])
        if T1w_path is not None:
            print("MRIs found for subject " + subject)
            print(filepaths)
            if T2w_path is None:
                print("Did not find T2w images for subject " + subject)
            subject_name = subject + "_reco"
            freesurfer_commands_for_subject = make_freesurfer_command_recon_all(T1w_path, T2w_path, subject_name)
            freesurfer_command_file.write(freesurfer_commands_for_subject)
            print("Recon-all commands made for subject " + subject_name +"\n")

    initcommand = init_freesurfer_command(freesurfer_home, reftep_reco_dir_aalto_fatsuppressed,nobash=True)
    freesurfer_command_file.write(initcommand)
    sub_dirs = os.listdir(subjects_path_aalto)
    for subject in sub_dirs:
        if subject != '.DS_Store':
            T1w_path, T2w_path, mri_directory_correct = None, None, None #for control and non overlapping structs
            subject_path_now = subjects_path_aalto + subject + "/"
            session_types = os.listdir(subject_path_now)[1:]
            for session_type in session_types:
                directory_here = subject_path_now + session_type + "/"
                dirs_here = os.listdir(directory_here)
                if 'anat' in dirs_here and 'dwi' in dirs_here:
                    mri_dirs_in_mri_dir = dirs_here
                    mri_directory = directory_here
            if mri_dirs_in_mri_dir is not None:
                print("MRIs found for " + subject)
                for mri_dir in mri_dirs_in_mri_dir:
                    if mri_dir == "anat":
                        anat_dir = mri_directory + mri_dir + "/"
                        print(anat_dir)
                        for recording_type in os.listdir(anat_dir):
                            if "T1w.nii" in recording_type and "fatsuppressed" in recording_type:
                                cdir = "".join([subjects_path_aalto ,"".join(anat_dir.split("BIDS")[1][1:])])
                                T1w_path = cdir + recording_type
                            if "T2w.nii" in recording_type:
                                cdir = "".join([subjects_path_aalto ,"".join(anat_dir.split("BIDS")[1][1:])])
                                T2w_path = cdir + recording_type
                if T1w_path is not None and T2w_path is not None:
                    subject_name = subject.split("-")[0] + "-" + subject.split("-")[1][-3:]+ "_fatsuppressed_reco"
                    freesurfer_commands_for_subject = make_freesurfer_command_recon_all(T1w_path, T2w_path, subject_name)
                    freesurfer_command_file.write(freesurfer_commands_for_subject)
                    print("Recon-all commands made for subject " + subject_name +" with fatsuppressed T1\n")
            else:
                print("Did not find MRIs for "+ subject)
    freesurfer_command_file.close()